In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/ga_hits.csv')
df.head()


,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,NaN
1,7750352294969115059.1640271109.1640271109,2021-12-23,597331.0,41,event,NaN,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show,NaN,NaN
2,885342191847998240.1640235807.1640235807,2021-12-23,796252.0,49,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show,NaN,NaN
3,142526202120934167.1640211014.1640211014,2021-12-23,934292.0,46,event,NaN,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show,NaN,NaN
4,3450086108837475701.1640265078.1640265078,2021-12-23,768741.0,79,event,NaN,sberauto.com/cars/all/mercedes-benz/cla-klasse...,quiz,quiz_show,NaN,NaN


In [3]:
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15726470 entries, 0 to 15726469
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      object 
 1   hit_date        object 
 2   hit_time        float64
 3   hit_number      int64  
 4   hit_type        object 
 5   hit_referer     object 
 6   hit_page_path   object 
 7   event_category  object 
 8   event_action    object 
 9   event_label     object 
 10  event_value     float64
dtypes: float64(2), int64(1), object(8)
memory usage: 1.3+ GB
None
(15726470, 11)


Целевое действие — события типа «Оставить заявку» и «Заказать звонок»
(ga_hits.event_action in ['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']).

In [4]:
df.event_action.value_counts()

event_action
view_card                     3558985
view_new_card                 3551009
sub_landing                   1798117
go_to_car_card                 973666
sub_view_cars_click            791515
                               ...   
click_on_android                    1
new_year_road_help_btn              1
titul_page_view                     1
show_pos_status_declined            1
survey_buy_after_sell_when          1
Name: count, Length: 230, dtype: int64

In [5]:
print('Количество незаполненных session_id: ',df.session_id.isna().sum())
print('Количество незаполненных event_action: ',df.event_action.isna().sum())

Количество незаполненных session_id:  0
Количество незаполненных event_action:  0


In [6]:
#Оставим только нужные столбцы
hits= df.copy()
hits = hits[['session_id', 'event_action']]
hits.head(5)

,session_id,event_action
0,5639623078712724064.1640254056.1640254056,quiz_show
1,7750352294969115059.1640271109.1640271109,quiz_show
2,885342191847998240.1640235807.1640235807,quiz_show
3,142526202120934167.1640211014.1640211014,quiz_show
4,3450086108837475701.1640265078.1640265078,quiz_show


In [7]:
# Трансформируем столбец в удобный нам формат: 0 для нецелевых действий и 1 для целевых
def event_action_transform(event_action):
    target_list= ['sub_car_claim_click', 'sub_car_claim_submit_click',
    'sub_open_dialog_click', 'sub_custom_question_submit_click',
    'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
    'sub_car_request_submit_click']
    if event_action in target_list:
        event_action = 1
    else:
        event_action = 0
    return event_action

hits['target'] = hits['event_action'].apply(event_action_transform)
hits.head()

,session_id,event_action,target
0,5639623078712724064.1640254056.1640254056,quiz_show,0
1,7750352294969115059.1640271109.1640271109,quiz_show,0
2,885342191847998240.1640235807.1640235807,quiz_show,0
3,142526202120934167.1640211014.1640211014,quiz_show,0
4,3450086108837475701.1640265078.1640265078,quiz_show,0


In [8]:
hits_clean = hits.drop(['event_action'], axis =1)
print('Количество значений: ', hits_clean.target.value_counts())

Количество значений:  target
0    15621562
1      104908
Name: count, dtype: int64


In [9]:
#Сгруппируем датасет по session_id: если хоть в каком-то из событий сессии целевое действие было =1, берем 1. Иначе 0

grouped_hits = hits_clean.groupby('session_id').max()

In [10]:
print('Размер полученного датасета:', grouped_hits.shape)
grouped_hits.head()

Размер полученного датасета: (1734610, 1)


,target
session_id,
1000009318903347362.1632663668.1632663668,0
1000010177899156286.1635013443.1635013443,0
1000013386240115915.1635402956.1635402956,0
1000017303238376207.1623489300.1623489300,0
1000020580299877109.1624943350.1624943350,0


In [11]:
grouped_hits.target.value_counts()

target
0    1683850
1      50760
Name: count, dtype: int64

In [12]:
#Сохраним датасет
grouped_hits.to_csv('data/grouped_hits.csv')

## Датасет sessions

In [13]:
#Второй датасет
df2 = pd.read_csv('data/ga_sessions.csv', sep = ',', dtype = {'client_id':'string'})
df2.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow


In [14]:
df_all = df2.merge(grouped_hits, on='session_id', how='inner')
df_all.to_csv('data/df_all.csv')


In [15]:
#Для пайплайна сделаем балансировку датасета (в блокноте с анализом будем рассматривать весь датасет целиком):
df_1=df_all[df_all['target'] == 1]
df_0=df_all[df_all['target'] == 0]
df_0=df_0.iloc[:50314] 
df_balanced = pd.concat([df_1, df_0], axis=0)
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)
df_balanced.to_csv('data/df_balanced.csv')
df=pd.read_csv('data/df_balanced.csv', index_col=0)
df.head()


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
0,971966755877048547.1637162829.1637162829,2.263036e+08,2021-11-17,18:27:09,2,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,lOMpyAuAISPFiAxjXIYM,desktop,Windows,NaN,NaN,1280x1024,YaBrowser,Russia,Surgut,0
1,9123087829188376033.1635153444.1635153444,2.124134e+09,2021-10-25,12:17:24,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,NaN,412x892,Chrome,Russia,Samara,0
2,9139487749489396109.1633257875.1633257875,2.127953e+09,2021-10-03,13:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Samsung,NaN,412x869,Chrome,Russia,(not set),0
3,9222207367917414622.1639262433.1639262433,2.147212e+09,2021-12-12,01:40:33,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,pHqgLcdBQPSLcrTpLkdr,mobile,Android,Samsung,NaN,412x892,Samsung Internet,Russia,Saint Petersburg,0
4,1268651693542320798.1632217757.1632217762,2.953810e+08,2021-09-21,12:00:00,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,375x812,Safari,Russia,Domodedovo,1


In [17]:
df.shape

(100628, 19)